# Работа со строковыми значениями

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Работа со строковыми значениям"
* https://pyformat.info/
* https://docs.python.org/3/library/re.html
    * https://docs.python.org/3/library/re.html#flags
    * https://docs.python.org/3/library/re.html#functions
* https://pythonru.com/primery/primery-primeneniya-regulyarnyh-vyrazheniy-v-python
* https://kanoki.org/2019/11/12/how-to-use-regex-in-pandas/
* https://realpython.com/nltk-nlp-python/

## Лабораторная работа 6

In [139]:
import pandas as pd
from bs4 import BeautifulSoup
import re
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

True

### Форматирование строк

1\. Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в виде `pd.DataFrame` `recipes` При помощи форматирования строк выведите информацию об id рецепта и времени выполнения 5 случайных рецептов в виде таблицы следующего вида:

    
    |      id      |  minutes  |
    |--------------------------|
    |    61178     |    65     |
    |    202352    |    80     |
    |    364322    |    150    |
    |    26177     |    20     |
    |    224785    |    35     |
    
Обратите внимание, что ширина столбцов заранее неизвестна и должна рассчитываться динамически, в зависимости от тех данных, которые были выбраны.

In [82]:
recipes = pd.read_csv('data/recipes_sample.csv')
recipe_sample = recipes.sample(5)

titles = f'|{"id":^15}|{"minutes":^15}|'
print(titles)

print(f"{'|-':-<{len(titles) - 1}}|")

for _, row in recipe_sample.iterrows():
    print(f"|{row['id']:^15}|{row['minutes']:^15}|")

|      id       |    minutes    |
|-------------------------------|
|    207371     |      80       |
|    350758     |      80       |
|    521196     |      60       |
|     55345     |      550      |
|    180849     |      75       |


2\. Напишите функцию `show_info`, которая по данным о рецепте создает строку (в смысле объекта python) с описанием следующего вида:

```
"Название Из Нескольких Слов"

1. Шаг 1
2. Шаг 2
----------
Автор: contributor_id
Среднее время приготовления: minutes минут
```

    
Данные для создания строки получите из файлов `recipes_sample.csv` (__ЛР2__) и `steps_sample.xml` (__ЛР3__). 
Вызовите данную функцию для рецепта с id `170895` и выведите (через `print`) полученную строку на экран.

In [123]:
def show_info(name, steps, minutes, author_id):
    result = f'"{name.title()}"\n\n'

    for index, step in enumerate(steps):
        result += f'{index + 1}. {step.capitalize()}\n'

    result += '----------\n'
    result += f'Автор: {author_id}\n'
    result += f'Среднее время приготовления: {minutes} минут\n'

    return result

with open('data/steps_sample.xml') as file:
    data = BeautifulSoup(file, 'xml')

recipe_id = 170895
recipe = recipes[recipes['id'] == recipe_id]
recipe_steps = []

for current_recipe in data.recipes.find_all('recipe'):
    if current_recipe.find('id').text == str(recipe_id):
        recipe_steps = [step.text for step in current_recipe.steps.find_all('step')]

print(show_info(recipe['name'].values[0], recipe_steps, recipe['minutes'].values[0], recipe['contributor_id'].values[0]))

"Leeks And Parsnips  Sauteed Or Creamed"

1. Clean the leeks and discard the dark green portions
2. Cut the leeks lengthwise then into one-inch pieces
3. Melt the butter in a medium skillet , med
4. Heat
5. Add the garlic and fry 'til fragrant
6. Add leeks and fry until the leeks are tender , about 6-minutes
7. Meanwhile , peel and chunk the parsnips into one-inch pieces
8. Place in a steaming basket and steam 'til they are as tender as you prefer
9. I like them fork-tender
10. Drain parsnips and add to the skillet with the leeks
11. Add salt and pepper
12. Gently sautee together for 5-minutes
13. At this point you can serve it , or continue on and cream it:
14. In a jar with a screw top , add the half-n-half and arrowroot
15. Shake 'til blended
16. Turn heat to low under the leeks and parsnips
17. Pour in the arrowroot mixture , stirring gently as you pour
18. If too thick , gradually add the water
19. Let simmer for a couple of minutes
20. Taste to adjust seasoning , probably an addi

In [37]:
print(show_info(
        name="george s at the cove black bean soup",
        steps=[
            "clean the leeks and discard the dark green portions",
            "cut the leeks lengthwise then into one-inch pieces",
            "melt the butter in a medium skillet , med",
        ],
        minutes=90,
        author_id=35193,
    ))

"George S At The Cove Black Bean Soup"

1. Clean the leeks and discard the dark green portions
2. Cut the leeks lengthwise then into one-inch pieces
3. Melt the butter in a medium skillet , med
----------
Автор: 35193
Среднее время приготовления: 90 минут



In [38]:
assert (
    show_info(
        name="george s at the cove black bean soup",
        steps=[
            "clean the leeks and discard the dark green portions",
            "cut the leeks lengthwise then into one-inch pieces",
            "melt the butter in a medium skillet , med",
        ],
        minutes=90,
        author_id=35193,
    )
    == '"George S At The Cove Black Bean Soup"\n\n1. Clean the leeks and discard the dark green portions\n2. Cut the leeks lengthwise then into one-inch pieces\n3. Melt the butter in a medium skillet , med\n----------\nАвтор: 35193\nСреднее время приготовления: 90 минут\n'
)

## Работа с регулярными выражениями

3\. Напишите регулярное выражение, которое ищет следующий паттерн в строке: число (1 цифра или более), затем пробел, затем слова: hour или hours или minute или minutes. Произведите поиск по данному регулярному выражению в каждом шаге рецепта с id 25082. Выведите на экран все непустые результаты, найденные по данному шаблону.

In [122]:
pattern = r'\d+\s(?:hours|hour|minutes|minute)'
recipe_id = 25082

recipe_steps = []

for current_recipe in data.recipes.find_all('recipe'):
    if current_recipe.find('id').text == str(recipe_id):
        recipe_steps = [step.text for step in current_recipe.steps.find_all('step')]

res = []

for step in recipe_steps:
    res += re.findall(pattern, step)

print(res)

['20 minutes', '10 minutes', '2 hours', '10 minutes', '20 minutes', '30 minutes']


4\. Напишите регулярное выражение, которое ищет шаблон вида "this..., but" _в начале строки_ . Между словом "this" и частью ", but" может находиться произвольное число букв, цифр, знаков подчеркивания и пробелов. Никаких других символов вместо многоточия быть не может. Пробел между запятой и словом "but" может присутствовать или отсутствовать.

Используя строковые методы `pd.Series`, выясните, для каких рецептов данный шаблон содержится в тексте описания. Выведите на экран количество таких рецептов и 3 примера подходящих описаний (текст описания должен быть виден на экране полностью).

In [108]:
pattern = r'^this[\w\s]*,\s?but'
recipes['description'] = recipes['description'].fillna(str())
matching_recipes = recipes['description'].str.contains(pattern)

print(f'Number of matched recipes: {len(recipes[matching_recipes])}\n')

description_sample = recipes[matching_recipes]['description'].sample(3)

for description in description_sample:
    print(f'Description: {description}\n')

Number of matched recipes: 134

Description: this has a lot of ingredients, but is totally easy to make.  an excellent home cooked meal that tastes like chinese take-out!  you can also make these as appetizers (as they are served at the restaurants such as pf changs).

Description: this is a very simple recipe, but it's made savory by using chicken broth and adding the cashews.  it doesn't take long to make, and i've found that fresh parsley really enhances the flavor of the dish rather than dried parsley.

Description: this has a lot of ingredients in it, but after a lot of experimenting, i've found that this makes a really wonderful combination for a chili.  i make this at least once a month and my bf and i enjoy it every time.  so i wanted to share it with the zaar community and also put it here for safe-keeping.  enjoy!



5\. В текстах шагов рецептов обыкновенные дроби имеют вид "a / b". Используя регулярные выражения, уберите в тексте шагов рецепта с id 72367 пробелы до и после символа дроби. Выведите на экран шаги этого рецепта после их изменения.

In [121]:
recipe_id = 72367

recipe_steps = []

for current_recipe in data.recipes.find_all('recipe'):
    if current_recipe.find('id').text == str(recipe_id):
        recipe_steps = [step.text for step in current_recipe.steps.find_all('step')]

for index in range(len(recipe_steps)):
    recipe_steps[index] = re.sub(r'\s*/\s*', '/', recipe_steps[index])
    print(recipe_steps[index])

mix butter , flour , 1/3 c
sugar and 1-1/4 t
vanilla
press into greased 9" springform pan
mix cream cheese , 1/4 c
sugar , eggs and 1/2 t
vanilla beating until fluffy
pour over dough
combine apples , 1/3 c
sugar and cinnamon
arrange on top of cream cheese mixture and sprinkle with almonds
bake at 350 for 45-55 minutes , or until tester comes out clean


### Сегментация текста

6\. Разбейте тексты шагов рецептов на слова при помощи пакета `nltk`. Посчитайте и выведите на экран кол-во уникальных слов среди всех рецептов. Словом называется любая последовательность алфавитных символов (для проверки можно воспользоваться `str.isalpha`). При подсчете количества уникальных слов не учитывайте регистр.

In [130]:
words = set()
for current_recipe in data.recipes.find_all('recipe'):
    recipe_steps = ' '.join([step.text for step in current_recipe.steps.find_all('step')])
    [words.update([word.lower()]) for word in nltk.word_tokenize(recipe_steps) if word.isalpha()]

len(set(words))

14926

7\. Разбейте описания рецептов из `recipes` на предложения при помощи пакета `nltk`. Найдите 5 самых длинных описаний (по количеству _предложений_) рецептов в датасете и выведите строки фрейма, соответствующие этим рецептами, в порядке убывания длины.

In [132]:
n_sentences = []

for description in recipes['description']:
    n_sentences += [len(nltk.sent_tokenize(description))]

recipes['n_sentences'] = n_sentences

longest = recipes.nlargest(5, 'n_sentences')

for _, row in longest.iterrows():
    print(row)

name                   my favorite buttercream icing for decorating
id                                                           334113
minutes                                                          30
contributor_id                                               681465
submitted                                                2008-10-30
n_steps                                                        12.0
description       this wonderful icing is used for icing cakes a...
n_ingredients                                                   NaN
n_sentences                                                      76
Name: 18408, dtype: object
name              alligator claws  avocado fritters  with chipot...
id                                                           287008
minutes                                                          45
contributor_id                                               765354
submitted                                                2008-02-19
n_steps              

8\. Напишите функцию, которая для заданного предложения выводит информацию о частях речи слов, входящих в предложение, в следующем виде:
```
PRP   VBD   DT      NNS     CC   VBD      NNS        RB   
 I  omitted the raspberries and added strawberries instead
``` 
Для определения части речи слова можно воспользоваться `nltk.pos_tag`.

Проверьте работоспособность функции на названии рецепта с id 241106.

Обратите внимание, что часть речи должна находиться ровно посередине над соотвествующим словом, а между самими словами должен быть ровно один пробел.


In [148]:
def sentence_pos(sentence):
    tags = nltk.pos_tag(nltk.tokenize.word_tokenize(sentence))

    pos = str()
    words = str()

    for tag in tags:
        max_length = max(len(tag[0]), len(tag[1]))

        pos += f'{tag[1]}'.center(max_length) + ' '
        words += f'{tag[0]}'.center(max_length) + ' '

    print(pos)
    print(words)

sentence_pos(recipes[recipes['id'] == 241106]['name'].values[0])

   JJ     NNS    IN     NNS    VBP    JJ    CC   JJ   NNS   
eggplant steaks with chickpeas feta cheese and black olives 
